In [1]:
import numpy as np
import pandas as pd
movies=pd.read_csv("/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings=pd.read_csv("/ratings.csv",usecols=['userId','movieId','rating'])
movies=pd.read_csv("/movies.csv",usecols=['movieId','title'])
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [8]:
ratings.shape

(100836, 3)

In [9]:
movies.shape

(9742, 2)

In [2]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [4]:
movies.describe().T

,count,mean,std,min,25%,50%,75%,max
movieId,9742.0,42200.353623,52160.494854,1.0,3248.25,7300.0,76232.0,193609.0


In [13]:
movie_users=ratings.pivot(index='movieId',columns='userId',values='rating').fillna(0)
movie_users.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from scipy.sparse import csr_matrix
movie_users_matrix=csr_matrix(movie_users.values)
movie_users_matrix

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [15]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(metric='cosine',algorithm='brute', n_neighbors=20)
model.fit(movie_users_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [18]:
!pip install fuzzywuzzy

In [19]:
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [41]:
def movie_recommender(movie_name,data,n):
    idx=process.extractOne(movie_name,movies['title'])[2]
    print('Movie Selected: ',movies['title'][idx],'Index: ',idx)
    print('Searching for recommendations.....')
    distances,indices=model.kneighbors(data[idx],n_neighbors=n)
    print(distances , indices)
    for i in indices:
        print(movies['title'][i].where(i!=idx))

In [42]:
movie_recommender('Toy Story', movie_users_matrix , 10)

Movie Selected:  Toy Story (1995) Index:  0
Searching for recommendations.....
[[0.         0.42739874 0.4343632  0.43573831 0.44261183 0.45290409
  0.45885465 0.4589107  0.46108723 0.46583124]] [[   0 2353  418  615  224  314  322  910  546  963]]
0                                                     NaN
2353                                 'night Mother (1986)
418                                  Jurassic Park (1993)
615                  Independence Day (a.k.a. ID4) (1996)
224             Star Wars: Episode IV - A New Hope (1977)
314                                   Forrest Gump (1994)
322                                 Lion King, The (1994)
910     Once Upon a Time in the West (C'era una volta ...
546                            Mission: Impossible (1996)
963                                           Diva (1981)
Name: title, dtype: object


In [43]:
movie_recommender('iron man', movie_users_matrix , 10)

Movie Selected:  Iron Man (2008) Index:  6743
Searching for recommendations.....
[[7.77156117e-16 4.09144753e-01 4.13091640e-01 4.18127003e-01
  4.35628075e-01 4.58178695e-01 4.59801664e-01 4.72782777e-01
  4.85689712e-01 4.86523926e-01]] [[6743 7197 7195 7354 6726 7137 7026 7571 3880 6388]]
6743                                            NaN
7197                                  Garage (2007)
7195                        Merry Madagascar (2009)
7354                             A-Team, The (2010)
6726                         Superhero Movie (2008)
7137                         Thirst (Bakjwi) (2009)
7026                                 Scorpio (1973)
7571                                 Win Win (2011)
3880                  Look Who's Talking Now (1993)
6388    After the Wedding (Efter brylluppet) (2006)
Name: title, dtype: object
